# Predicting Blight Violations

_This project was completed as a part of the Applied Data Science with Python Specialization from Coursera._

In this project, we will evaluate the performance and predictive power of a model that has been trained and tested on data collected from Blight Violation Notices (BVN), or Blight Tickets, that have been issued to property owners who have violated City of Detroit ordinances that govern how property owners must maintain the exterior of their property. Blight Tickets are issued by city inspectors, police officers, neighborhood city hall managers and other city officials who investigate complaints of blight and are managed by the Department of Administrative Hearings.

The target variable is compliance, which is __True__ if the ticket was paid early, on time, or within one month of the hearing data, __False__ if the ticket was paid after the hearing date or not at all, and __Null__ if the violator was found not responsible. Compliance is not avaliable in the dataset so it must be calculated.

The dataset for this project originates from the City of Detroit's Open Data Portal initiative and is updated daily. For this project the dataset spans tickets issued from March 2004 to March 2018. The dataset is avaliable at: https://data.detroitmi.gov/Property-Parcels/Blight-Violations/ti6p-wcg4

## Pre-Processing

Imports the raw data, calculates compliance for each ticket, removes features that cause data leakage, and seperates the data into two dataset based on when the violation occured. Violations issused before 2017 will be used to train and test the model, and tickets issued on and after 2017 will be used to validate the model.

In [4]:
# Common imports
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import normalize
from scipy.sparse import coo_matrix, hstack

# To make the notebook's output stable across runs
random_seed = 12062017
np.random.seed(random_seed)

In [16]:
def load_blight_raw(blight_path):
    '''
    Loads the raw blight_violations.csv
    Returns a pandas dataframe.
    '''
    csv_path = os.path.join(blight_path, "Blight_Violations.csv")
    blight_raw = pd.read_csv(csv_path, encoding='ISO-8859-1')
    print("RAW Blight Violations dataset has {} observations with {} features each.".format(*blight_raw.shape))
    return blight_raw

def pre_processing(blight_df):
    ''' 
    Takes blight panda dataframe and makes column names more reabable, sets ticket_id as index, creates compliance (the 
    prediction variable), and creates compliance_details (explains why a tickets was labeled as complantent or non-complantent).
    Returns a panda dataframe.
    '''
    # Makes column names more reabable
    # Gets columns from dataframe
    columns_names = pd.Series(blight_df.columns)
    # Removes parentheses, removes right spaces
    columns_names = columns_names.str.split('(').str[0].str.strip()
    # Coverts column names to lowercase and replaces spaces with underscores
    columns_names = columns_names.str.lower().str.replace(' ', '_')
    blight_df.columns = columns_names
    # Sets ticket_id as index
    blight_df.set_index('ticket_id', inplace=True)
    # Defines prediction variable compliance and helper variable compliance_detail
    # The values of these variables are NOT correct
    blight_df['compliance'] = 0
    blight_df['compliance_detail'] = np.NaN    
    return blight_df

def clean_up(blight_df):
    # Removes instances where 'ticket_issued_date' is not in (2000, 2017) 
    blight_df = blight_df[blight_df['violation_date'].str.contains('[0-9]{2}/[0-9]{2}/[20]{2}[0-9]{2}') == True]
    # Converts 'ticket_issued_date' to datetime
    blight_df.loc[:, 'violation_date'] = pd.to_datetime(blight_df['violation_date'])
    # Converts 'payment_date' to datetime
    blight_df.loc[:, 'payment_date'] = pd.to_datetime(blight_df['payment_date'].str.extract('([0-9]{2}/[0-9]{2}/20{1}[0-9]{2})')) 
    # Converts bad values in 'ticket_issued_time' to NaNs (Dataset error removed by publisher)
    #blight_df.loc[blight_df['violation_date'] == '00000000000000.000', 'violation_date'] = np.nan
    # Converts 'ticket_issued_time' to datetime.time
    blight_df.loc[:, 'ticket_issued_time'] = pd.DatetimeIndex(blight_df['ticket_issued_time']).time
    # Converts 'hearing_time' to datetime.time
    blight_df.loc[:, 'hearing_time'] = pd.DatetimeIndex(blight_df['hearing_time']).time
    # Remove "$" from 'judgement_amount'
    blight_df.loc[:, 'judgment_amount'] = blight_df['judgment_amount'].str.strip('$')    
    # Converts 'judgment_amount' to float    
    blight_df.loc[:, 'judgment_amount'] = blight_df['judgment_amount'].astype(float)
    # Removes "$" from 'payment_amount' and converts it to float
    blight_df.loc[:, 'payment_amount'] = blight_df['payment_amount'].str.strip('$').astype(float)
    return blight_df

def null_compliance(blight_df):
    '''
    Compliance = np.NaN
    Tickets that cannot be classified as compliant or non-compliant because they were ruled as not responsible in disposition
    or they are awaiting judgement.
    Returns a pandas dataframe
    '''
    # Dispositions that are ruled as not responsible or are still pending
    null_dispositions = ['Not responsible by Dismissal', 'Not responsible by City Dismissal', 'PENDING JUDGMENT', 
                 'Not responsible by Determination','SET-ASIDE (PENDING JUDGMENT)']
    # Loops over dispositions and sets 'compliance' values to np.NaN
    # Loops over dispositions and sets 'compliance_detail' values to 'Not Responsible/Pending Judgement'
    for dispositions in null_dispositions:
        blight_df.loc[blight_df['disposition'] == dispositions, 'compliance'] = np.NaN
        blight_df.loc[blight_df['disposition'] == dispositions, 'compliance_detail'] = 'Not Responsible/Pending Judgement'
    return blight_df

def compliant(blight_df):
    '''
    Compliance = 1
    Tickets that are classified as compliant because they had no fine, fine was waved, made a payment with hearing pending,
    early payment (hearing not pending), payment on time, or payment within one month after hearing date.
    Returns a pandas dataframe
    '''
    ## Compliant by no fine
    # Fine Waived by Determintation
    blight_df.loc[blight_df['disposition'] == 'Responsible (Fine Waived) by Determination', 'compliance'] = 1
    blight_df.loc[blight_df['disposition'] == 'Responsible (Fine Waived) by Determination', 'compliance_detail'] = 'Compliant by no fine'
    # Fine Waived by Admission
    blight_df.loc[blight_df['disposition'] == 'Responsible (Fine Waived) by Admission', 'compliance'] = 1
    blight_df.loc[blight_df['disposition'] == 'Responsible (Fine Waived) by Admission', 'compliance_detail'] = 'Compliant by no fine'
    ## Compliant by Payment
    # Payment with PENDING hearing
    blight_df.loc[(
        (blight_df['hearing_date'] == 'PENDING') &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance'] = 1
    blight_df.loc[(
        (blight_df['hearing_date'] == 'PENDING') &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance_detail'] = 'Compliant by payment with PENDING hearing'
    # Transforms values in 'hearing_date' and 'payment_date' to panda date types
    dummy_hearing_date = blight_df['hearing_date'].copy()
    blight_df.loc[blight_df['hearing_date'] == 'PENDING', 'hearing_date'] = np.nan
    blight_df.loc[:, 'hearing_date'] = pd.to_datetime(blight_df['hearing_date'].str.extract('([0-9]{2}/[0-9]{2}/20{1}[0-9]{2})'))
    # Early Payment, payment before hearing date
    blight_df.loc[(
        (blight_df['payment_date'] < blight_df['hearing_date']) &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance'] = 1
    blight_df.loc[(
        (blight_df['payment_date'] < blight_df['hearing_date']) &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance_detail'] = 'Compliant by early payment'
    # Payment on time, payment on hearing date
    blight_df.loc[(
        (blight_df['payment_date'] == blight_df['hearing_date']) &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance'] = 1
    blight_df.loc[(
        (blight_df['payment_date'] == blight_df['hearing_date']) &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance_detail'] = 'Compliant by payment on time'
    # Payment within one month after hearing date
    blight_df.loc[(
        ((blight_df['payment_date'] - blight_df['hearing_date'])/np.timedelta64(1, 'M') <= 1.0) &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance'] = 1
    blight_df.loc[(
        ((blight_df['payment_date'] - blight_df['hearing_date'])/np.timedelta64(1, 'M') <= 1.0) &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance_detail'] = 'Compliant by payment within 1 Month'
    # Sets 'hearing_date' to orginal state
    blight_df.loc[:, 'hearing_date'] = dummy_hearing_date
    return blight_df

def non_compliant(blight_df):
    '''
    Compliance = 0
    Tickets that are classified as non-compliant because they made no payment or a payment after one month (late payment).
    Returns a pandas dataframe
    '''
    # Transforms values in 'hearing_date' to panda date types
    dummy_hearing_date = blight_df['hearing_date'].copy()
    blight_df.loc[blight_df['hearing_date'] == 'PENDING', 'hearing_date'] = np.nan
    blight_df.loc[:, 'hearing_date'] = pd.to_datetime(blight_df['hearing_date'].str.extract('([0-9]{2}/[0-9]{2}/20{1}[0-9]{2})'))
    # Non-compliant by late payment
    blight_df.loc[(
        ((blight_df['payment_date'] - blight_df['hearing_date'])/np.timedelta64(1, 'M') > 1.0) &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance_detail'] = 'Non-compliant by late payment more than 1 month'
    # Non-compliant by no payment
    blight_df.loc[blight_df['compliance_detail'].isnull(), 'compliance_detail'] = 'Non-compliant by no payment'
    # Sets 'hearing_date' to orginal state
    blight_df.loc[:, 'hearing_date'] = dummy_hearing_date
    return blight_df

def populate_compliance(blight_df):
    '''
    Populates compliance and compliance_details in blight_df with the correct values. 
    Returns a panda dataframe.
    '''
    blight_df = null_compliance(blight_df)
    blight_df = compliant(blight_df)
    blight_df = non_compliant(blight_df)  
    return blight_df

def remove_leakage(blight_df):
    ''' 
    In blight_df removes variables to prevent data leakage and variables with mostly NaNs,
    Returns a panda dataframe
    '''
    blight_df = blight_df[['agency_name', 'inspector_name', 'violator_name','violation_street_number', 'violation_street_name', 
                       'mailing_address_street_name', 'mailing_address_street_name', 'mailing_address_city', 
                       'mailing_address_state', 'mailing_address_zip_code', 'mailing_address_non-usa_code', 
                       'mailing_address_country', 'violation_date', 'hearing_date', 'violation_code', 'violation_description',
                       'disposition', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 
                       'judgment_amount', 'violation_latitude', 'violation_longitude', 'compliance']]  
    return blight_df 

def process_blight(blight_path, pre_process):
    '''
    If pre_process is true - Loads the raw blight_violations.csv, cleans the data, computes compliance, removes features with 
    data leakage and saves this dataframe. If pre_process is False, then it loads the pre-processed data.
    Returns a pandas dataframe.
    '''
    if pre_process == False:
        blight_df = load_blight_raw(blight_path)
        blight_df = pre_processing(blight_df)
        blight_df = clean_up(blight_df)
        blight_df = populate_compliance(blight_df)
        blight_df = remove_leakage(blight_df)
        save_blight_data(blight_path, blight_df)
    else:
        blight_df = load_blight(blight_path)
    print("PROCESSED Blight Violations dataset has {} observations with {} features each.".format(*blight_df.shape))        
    return blight_df

def save_blight_data(blight_path, blight_df):
    '''
    Saves the processed blight_df
    Returns nothing.
    '''
    csv_path = os.path.join(blight_path, "Blight_Violations_Processed.csv")
    blight_df.to_csv(csv_path)
    
def load_blight(blight_path):
    '''
    Loads the preprocessed blight_violations.csv
    Returns a pandas dataframe.
    '''
    csv_path = os.path.join(blight_path, "Blight_Violations_Processed.csv")
    blight_df = pd.read_csv(csv_path)
    blight_df.set_index('ticket_id', inplace=True)
    return blight_df

In [19]:
# Loads the raw blight data and pre-processes
blight_df = process_blight(r"C:\Users\Adrian\Desktop", False)
# Checks to make sure the data loaded properly
blight_df.info()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: DtypeWarning: Columns (8,34) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


RAW Blight Violations dataset has 373848 observations with 40 features each.


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:98: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:139: FutureWarning: currently extract(expand=None) means expa

PROCESSED Blight Violations dataset has 373844 observations with 26 features each.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 373844 entries, 47056 to 247933
Data columns (total 26 columns):
agency_name                     373844 non-null object
inspector_name                  373844 non-null object
violator_name                   373842 non-null object
violation_street_number         373844 non-null int64
violation_street_name           373779 non-null object
mailing_address_street_name     373841 non-null object
mailing_address_street_name     373841 non-null object
mailing_address_city            372028 non-null object
mailing_address_state           371565 non-null object
mailing_address_zip_code        372025 non-null object
mailing_address_non-usa_code    1819 non-null object
mailing_address_country         1830 non-null object
violation_date                  373844 non-null datetime64[ns]
hearing_date                    373234 non-null object
violation_code               